In [ ]:
!pip install --upgrade pip
!pip install numpy==1.24.4
!pip install opencv-python ultralytics tensorflow deep_sort_realtime tensorflow-hub
!pip  install SoccerNet


'!pip install --upgrade pip\n!pip install numpy==1.24.4\n!pip install opencv-python ultralytics tensorflow deep_sort_realtime tensorflow-hub\n!pip  install SoccerNet\n'

In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader

mySN = SoccerNetDownloader(LocalDirectory="SoccerNetData")
mySN.password = "s0cc3rn3t"

mySN.downloadGames(files=["1_224p.mkv", "2_224p.mkv"], split=["train","valid","test","challenge"])


In [ ]:
import cv2

cap = cv2.VideoCapture("SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_224p.mkv")

'\nwhile cap.isOpened():\n    ret, frame = cap.read()\n    if not ret:\n        break\n    cv2.imshow("Frame", frame)\n    if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n        break\ncap.release()\ncv2.destroyAllWindows()\n'

In [63]:
# YOLO
from ultralytics import YOLO
yolo_model = YOLO("yolov8n.pt")



Tracking com Deep SORT

In [64]:
from deep_sort_realtime.deepsort_tracker import DeepSort

tracker = DeepSort(max_age=30)


Pipeline completo com YOLO

In [65]:
# yolo_tracking.py
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=30)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
max_frames = 2*60*fps/3
frame_count=0

out = cv2.VideoWriter("yolo_tracking_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

while cap.isOpened():
    if frame_count >= max_frames:
        break
    else:
        frame_count +=1
        print(frame_count)
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)[0]
        detections = []
        for box in results.boxes:
            cls = int(box.cls[0])
            if cls in [0, 32]:  # pessoa ou bola
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, "object"))

        tracks = tracker.update_tracks(detections, frame=frame)
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"YOLO ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()

1

0: 384x640 17 persons, 62.7ms
Speed: 2.1ms preprocess, 62.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
2

0: 384x640 16 persons, 61.1ms
Speed: 2.1ms preprocess, 61.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
3

0: 384x640 15 persons, 69.9ms
Speed: 2.3ms preprocess, 69.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
4

0: 384x640 16 persons, 159.9ms
Speed: 6.0ms preprocess, 159.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
5

0: 384x640 14 persons, 101.9ms
Speed: 6.0ms preprocess, 101.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
6

0: 384x640 14 persons, 98.0ms
Speed: 3.0ms preprocess, 98.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
7

0: 384x640 13 persons, 102.7ms
Speed: 2.8ms preprocess, 102.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
8

0: 384x640 15 persons, 96.3ms
Speed: 2.1ms preprocess, 96.3ms inference, 1.2ms po

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

cap = cv2.VideoCapture("SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_224p.mkv")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
max_frames = int(2 * 60 * fps / 3)
frame_count_ssd = 0

ssd = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
tracker = DeepSort(max_age=30)
detector_input_size = 320

out = cv2.VideoWriter("ssd_tracking_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

while cap.isOpened():
    if frame_count_ssd >= max_frames:
        break

    ret, frame = cap.read()
    if not ret:
        break
    frame_count_ssd += 1

    img = cv2.resize(frame, (detector_input_size, detector_input_size))
    img_tensor = tf.convert_to_tensor(img, dtype=tf.uint8)[tf.newaxis, ...]

    output = ssd(img_tensor)
    boxes = output["detection_boxes"][0].numpy()
    class_ids = output["detection_classes"][0].numpy().astype(int)
    scores = output["detection_scores"][0].numpy()

    detections = []
    for i in range(len(scores)):
        if scores[i] < 0.5:
            continue
        cls_id = class_ids[i]
        if cls_id not in [1, 33]:  
            continue
        y1, x1, y2, x2 = boxes[i]
        x1, y1, x2, y2 = int(x1 * w), int(y1 * h), int(x2 * w), int(y2 * h)
        detections.append(([x1, y1, x2 - x1, y2 - y1], float(scores[i]), "object"))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(frame, f"SSD ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()
print("SSD vídeo salvo: ssd_tracking_output.mp4")


✅ SSD vídeo salvo: ssd_tracking_output.mp4
